In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List

from tqdm import tqdm

from spluslib import SplusService, ImageType
from xmatchlib import XTable, CrossMatch
from astromodule.io import load_table, save_table
from astromodule.tableops import stilts_crossmatch, stilts_unique
from astropy import units as u

# Prepare Dataset

In [2]:
df_part1_raw = load_table('tables/catalog_v4_part1+raw.csv', comment='#')
df_part2_raw = load_table('tables/catalog_v4_part2+raw.csv', comment='#')
df_part3_raw = load_table('tables/catalog_v4_part3+raw.csv', comment='#')
df_part3_allow = load_table('tables/catalog_v4_part3+allow.csv', comment='#')
df_part4_raw = load_table('tables/catalog_v4_part4+raw.csv', comment='#')
df_part4_deny = load_table('tables/catalog_v4_part4+deny.csv', comment='#')
df_v4_deny = load_table('tables/catalog_v4_deny.csv', comment='#')

In [3]:
columns = ['cluster_name', 'ra', 'dec', 'z']

df_part1 = df_part1_raw[columns]
df_part1['shard'] = [1] *len(df_part1)
df_part1['release'] = ['dr4'] * len(df_part1)

df_part2 = df_part2_raw[columns]
df_part2['shard'] = [2] *len(df_part2)
df_part2['release'] = ['dr4'] * len(df_part2)

df_part3 = df_part3_raw[df_part3_raw.cluster_name.isin(df_part3_allow.cluster_name.values)]
df_part3 = df_part3[columns]
df_part3['shard'] = [3] *len(df_part3)
df_part3['release'] = ['dr4'] * len(df_part3)

df_part4 = df_part4_raw[~df_part4_raw.cluster_name.isin(df_part4_deny.cluster_name.values)]
df_part4 = df_part4[columns]
df_part4['shard'] = [4] *len(df_part4)
df_part4['release'] = ['idr5'] * len(df_part4)

print('#PART_3_ALLOW:', len(df_part3_allow), ' |  #PART_3:', len(df_part3))
print('#PART_4_RAW - #PART_4_DENY:', len(df_part4_raw) - len(df_part4_deny), ' |  #PART_4:', len(df_part4))

#PART_3_ALLOW: 42  |  #PART_3: 31
#PART_4_RAW - #PART_4_DENY: 21  |  #PART_4: 22


### Concatenação dos pedaços

In [4]:
df_v4_raw = pd.concat((df_part1, df_part2, df_part3, df_part4))
print('len:', len(df_v4_raw), ' | ', 'columns:', df_v4_raw.columns.tolist())
save_table(df_v4_raw, 'tables/catalog_v4_raw.csv')

len: 146  |  columns: ['cluster_name', 'ra', 'dec', 'z', 'shard', 'release']


### Padronização dos nomes e remoção de nomes duplicados

In [5]:
df_v4 = df_v4_raw.copy()
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(' ', '')
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^ABELL0+', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^ABELL', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^Abell0+', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^Abell', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^A0+', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^AS', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^ACO', 'A', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^A(\d+)\w*', r'A\1', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^RXC', 'MCXC', regex=True)
df_v4['cluster_name'] = df_v4.cluster_name.str.replace(r'^MKW0+', 'MKW', regex=True)
df_v4 = df_v4.drop_duplicates('cluster_name', keep='last')

print('len:', len(df_v4))

len: 107


### Crossmatch para identificação de duplicados

In [6]:
df_v4_unique = stilts_unique(
  table=df_v4,
  radius=45*u.arcmin,
  action='identify'
)
df_v4_unique[df_v4_unique.GroupID > 0].sort_values('GroupID')

,cluster_name,ra,dec,z,shard,release,GroupID,GroupSize
0,A194,21.420000,-1.407200,0.01800,2,dr4,1,2
56,MCXCJ0125.6-0125,21.387800,-1.506900,0.01800,3,dr4,1,2
28,WHYJ034138-531028,55.412308,-53.180950,0.08520,2,dr4,2,2
29,A3158,55.727288,-53.640609,0.05970,2,dr4,2,2
38,PSZ2G272.88+19.14,157.429700,-35.330424,0.01110,2,dr4,3,2
66,NGC3258,157.148000,-35.642000,0.00930,3,dr4,3,2
51,Hydra,159.174160,-27.524440,0.01260,2,dr4,4,2
57,HCG048,159.440200,-27.080500,0.00900,3,dr4,4,2
59,MCXCJ1253.2-1522,193.310000,-15.380000,0.04620,3,dr4,5,2
61,A1631,193.201000,-15.433200,0.05080,3,dr4,5,2


In [7]:
df_v4 = df_v4[~df_v4.cluster_name.isin(df_v4_deny.cluster_name)]
print('len:', len(df_v4))

len: 73


### Criação dos indices

In [8]:
if 'cluster_id' not in df_v4.columns:
  df_v4.insert(0, 'cluster_id', range(1, len(df_v4) + 1))
save_table(df_v4, 'tables/catalog_v4.csv')
save_table(df_v4.sort_values('ra'), 'tables/catalog_v4_RA.csv')
save_table(df_v4.sort_values('dec'), 'tables/catalog_v4_DEC.csv')
df_v4

,cluster_id,cluster_name,ra,dec,z,shard,release
0,1,A194,21.42000,-1.40720,0.0180,2,dr4
1,2,A2457,338.92000,1.48490,0.0578,2,dr4
2,3,A267,28.17485,1.00709,0.2327,2,dr4
3,4,MCXCJ0458.9-0029,74.72958,-0.48917,0.0150,2,dr4
4,5,MCXCJ0920.0+0102,140.00208,1.04000,0.0175,2,dr4
...,...,...,...,...,...,...,...
19,69,A1437,180.10583,3.33361,0.1345,4,idr5
25,70,MKW8,220.15916,3.47639,0.0270,4,idr5
29,71,A3809,326.74084,-43.91000,0.0623,4,idr5
35,72,A2415,331.41876,-5.59333,0.0581,4,idr5


In [9]:
def compute_radius(z):
  df_radius = pd.read_csv('tables/z_rad15mpc-degb.dat', sep=' ')
  return df_radius.iloc[(df_radius['z'] - z).abs().argsort()[:1]]['radius'].values[0]


def batch_download_cluster_region(
  clusters: List[str], 
  central_ra:List[float], 
  central_dec: List[float], 
  radius: List[float], 
):
  query_template = """
  SELECT 
    dual_g.RA, dual_g.DEC, dual_g.Field, 
    dual_g.g_auto, dual_r.r_auto, dual_i.i_auto, dual_u.u_auto, dual_z.z_auto, dual_r.r_aper_6,
    photoz.zml, photoz.odds, '{cluster}' AS cluster
  FROM
    dr4_dual_g AS dual_g
    INNER JOIN
    dr4_dual_i AS dual_i ON dual_i.ID = dual_g.ID
    INNER JOIN
    dr4_dual_r AS dual_r ON dual_r.ID = dual_g.ID
    INNER JOIN
    dr4_dual_u AS dual_u ON dual_u.ID = dual_g.ID
    INNER JOIN
    dr4_dual_z AS dual_z ON dual_z.ID = dual_g.ID
    INNER JOIN
    dr4_vacs.dr4_gal_photoz AS photoz ON photoz.ID = dual_g.ID
  WHERE
    r_auto BETWEEN {r_min} AND {r_max} AND
    1 = CONTAINS(
      POINT('ICRS', dual_g.RA, dual_g.DEC), 
      CIRCLE('ICRS', {ra}, {dec}, {radius})
    )
  """
  
  mag_ranges = [
    (13, 15), (15, 16), (16, 17), (17, 17.5), (17.5, 18), (18, 18.5), 
    (18.5, 19), (19, 19.5), (19.5, 20), (20, 20.5), (20.5, 21), (21, 21.5),
  ]
  
  for i in range(len(clusters)):
    _cluster = clusters[i]
    _central_ra = central_ra[i]
    _central_dec = central_dec[i]
    _radius = radius[i]
    
    print(f'[{i + 1} / {len(clusters)}] {_cluster}')
    
    save_path = Path('outputs_v4') / 'photo' / f'cluster_{_cluster}.csv'
    
    if save_path.exists(): continue
    
    query = [
      query_template.format(
        cluster=_cluster, 
        ra=_central_ra, 
        dec=_central_dec, 
        radius=_radius,
        r_min=r_min,
        r_max=r_max,
      )
      for (r_min, r_max) in mag_ranges
    ]

    # save_path = [
    #   Path('outputs_v4') / f'cluster_{_cluster}.csv' for _cluster in clusters
    # ]

    sp = SplusService(username='natanael', password='natan')
    sp.batch_query(query, save_path=save_path, replace=True, join=True, workers=5)


def batch_search_idr5(df_photo, df_clusters, replace=False):
  output_folder = Path('outputs_v4')
  columns = ['RA', 'DEC', 'zml', 'odds', 'field']
  for i, cluster in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    z = cluster['z']
    ra = cluster['ra']
    dec = cluster['dec']
    name = cluster['cluster_name']
    save_path = output_folder / 'photo' / f'cluster_{name}.csv'
    search_radius = compute_radius(z)
    pbar.set_description(f'{name} ({search_radius:.2f} deg)')
    if save_path.exists() and not replace: continue
    search_radius2 = search_radius ** 2
    df = df_photo[
      (((df_photo.RA - ra)**2 + (df_photo.DEC - dec)**2) < search_radius2)
    ]
    save_table(df[columns], save_path)



def batch_search_spec(df_spec, df_clusters, suffix, replace=False):
  Z_RANGE = 0.02
  output_folder = Path('outputs_v4')
  columns = ['RA', 'DEC', 'z', 'e_z']
  for i, cluster in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    z = cluster['z']
    ra = cluster['ra']
    dec = cluster['dec']
    name = cluster['cluster_name']
    save_path = output_folder / 'spec' / f'{name}_{suffix}.parquet'
    search_radius = compute_radius(z)
    pbar.set_description(f'{name} ({search_radius:.2f} deg)')
    if save_path.exists() and not replace: continue
    search_radius2 = search_radius ** 2
    df = df_spec[
      (((df_spec.RA - ra)**2 + (df_spec.DEC - dec)**2) < search_radius2) &
      df_spec['z'].between(z - Z_RANGE, z + Z_RANGE) &
      df_spec['f_z'].str.startswith('KEEP')
    ]
    save_table(df[columns], save_path)
    
    
    
def batch_crossmatch_spec_photo(df_clusters, variant, replace=False):
  output_folder = Path('outputs_v4')
  for i, cluster in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    name = cluster['cluster_name']
    save_path = output_folder / 'match' / f'{name}_match+{variant}.parquet'
    pbar.set_description(f'{name}')
    if save_path.exists() and not replace: continue
    df_spec = output_folder / 'spec' / f'{name}_spec+{variant}.parquet'
    df_photo = output_folder / 'photo' / f'cluster_{name}.csv'
    df = stilts_crossmatch(
      table1=df_spec,
      table2=df_photo,
      ra1='RA',
      dec1='DEC',
      ra2='RA',
      dec2='DEC',
      join='all1',
      radius=1*u.arcsec,
    )
    save_table(df, save_path)
    
    
    
def batch_prepare_tables(df_clusters, variant, replace=False):
  output_folder = Path('outputs_v4') / 'paulo' / variant / 'clusters'
  output_folder.mkdir(exist_ok=True, parents=True)
  for i, row in (pbar := tqdm(df_clusters.iterrows(), total=len(df_clusters))):
    name = row['cluster_name']
    clsid = row['cluster_id']
    release = row['release']
    pbar.set_description(name)
    save_path = output_folder / f'cluster_{str(clsid).zfill(4)}.dat'
    if save_path.exists() and not replace: continue
    if release == 'idr5':
      columns = ['RA_1', 'DEC_1', 'z', 'e_z', 'zml', 'odds']
      rename_map = {
        'RA_1': 'RA', 
        'DEC_1': 'DEC',
        'z': 'zspec',
        'e_z': 'zspec-err',
        'zml': 'z_phot', 
        'odds': 'z_phot_odds',
      }
      df = load_table(Path('outputs_v4') / 'match' / f'{name}_match+{variant}.parquet')
      df = df[columns]
      df = df.rename(columns=rename_map)
      df = df.fillna(-999)
      df['g_auto'] = [-999] * len(df)
      df['r_auto'] = [-999] * len(df)
      df['i_auto'] = [-999] * len(df)
      df['u_auto'] = [-999] * len(df)
      df['z_auto'] = [-999] * len(df)
    elif release == 'dr4':
      columns = [
        'RA_1', 'DEC_1', 'z', 'e_z', 'zml', 'odds', 
        'g_auto', 'r_auto', 'i_auto', 'u_auto', 'z_auto'
      ]
      rename_map = {
        'RA_1': 'RA', 
        'DEC_1': 'DEC',
        'z': 'zspec',
        'e_z': 'zspec-err',
        'zml': 'z_phot', 
        'odds': 'z_phot_odds',
        'g_auto': 'g_mag', 
        'r_auto': 'r_mag', 
        'i_auto': 'i_mag', 
        'u_auto': 'u_mag', 
        'z_auto': 'z_mag',
      }
      df = load_table(Path('outputs_v4') / 'match' / f'{name}_match+{variant}.parquet')
      df = df[columns]
      df = df.rename(columns=rename_map)
      df = df.fillna(-999)
    save_table(df, save_path)
  index_columns = ['cluster_id', 'cluster_name', 'ra', 'dec', 'z']
  index_rename = {
    'cluster_name': 'name',
    'cluster_id': 'clsid',
    'z': 'zspec',
    'ra': 'RA',
    'dec': 'DEC'
  }
  df_index = df_clusters[index_columns].rename(columns=index_rename)
  save_table(df_index, output_folder.parent / 'index.dat')
      



def concat_tables(paths, save_path):
  df = load_table(paths[0])
  
  for i in tqdm(range(1, len(paths))):
    df2 = pd.read_csv(paths[i])
    df = pd.concat((df, df2), ignore_index=True)
  
  df = df[df.columns.drop(list(df.filter(regex='Unnamed:*')))]
  save_table(df, save_path)

In [10]:
df_clusters = load_table('tables/catalog_v4.csv')
df_clusters = df_clusters[df_clusters.release=='dr4']
clusters = df_clusters['cluster_name'].to_list()
central_ra = df_clusters['ra'].to_list()
central_dec = df_clusters['dec'].to_list()
radius = [compute_radius(_z) for _z in df_clusters['z'].to_list()]
batch_download_cluster_region(
  clusters=clusters,
  central_ra=central_ra,
  central_dec=central_dec,
  radius=radius
)

[1 / 58] A194
[2 / 58] A2457
[3 / 58] A267
[4 / 58] MCXCJ0458.9-0029
[5 / 58] MCXCJ0920.0+0102
[6 / 58] HCG97
[7 / 58] CL1058+0137
[8 / 58] MCXCJ1121.7+0249
[9 / 58] IC1365
[10 / 58] ESO351-021
[11 / 58] A119
[12 / 58] A168
[13 / 58] MS0116.3-0115
[14 / 58] WHLJ012023-000444
[15 / 58] MCXCJ0229.3-3332
[16 / 58] NGC1132
[17 / 58] A3112
[18 / 58] A3122
[19 / 58] A3135
[20 / 58] Fornax
[21 / 58] MCXCJ0340-4542
[22 / 58] A3158
[23 / 58] A3223
[24 / 58] MCXCJ0413.9-3805
[25 / 58] A463
[26 / 58] A970
[27 / 58] MKW4
[28 / 58] A1644
[29 / 58] NGC5004
[30 / 58] A3733
[31 / 58] A3744
[32 / 58] RXJ2137.1+0026
[33 / 58] A2440
[34 / 58] A3880
[35 / 58] Hydra
[36 / 58] A3266
[37 / 58] A4038
[38 / 58] A3581
[39 / 58] A3128
[40 / 58] A1631
[41 / 58] WBL074
[42 / 58] A1750
[43 / 58] A4049
[44 / 58] NGC3258
[45 / 58] WBL288
[46 / 58] MKW1
[47 / 58] [YMV2007]3820
[48 / 58] [YMV2007]49
[49 / 58] [YMV2007]4713
[50 / 58] WBL550
[51 / 58] [YMV2007]7604
[52 / 58] [YMV2007]7609
[53 / 58] WBL081
[54 / 58] A301


In [12]:
df_clusters = load_table('tables/catalog_v4.csv')
df_clusters = df_clusters[df_clusters.release=='idr5']
df_photo = load_table('/mnt/hd/natanael/astrodata/idr5_photoz/idr5_photoz.parquet')
batch_search_idr5(
  df_clusters=df_clusters,
  df_photo=df_photo,
  replace=False,
)

A4059 (4.37 deg): 100%|██████████| 15/15 [00:00<00:00, 67.12it/s]


In [13]:
df_clusters = load_table('tables/catalog_v4.csv')
df_spec = load_table('tables/SpecZ_Catalogue_20231101.parquet')
df_spec_GU = df_spec[
  df_spec.class_spec.str.startswith('GALAXY') | 
  df_spec.class_spec.str.startswith('UNCLEAR')
].copy(deep=True)
df_spec_G = df_spec_GU[
  df_spec_GU.class_spec.str.startswith('GALAXY')
].copy(deep=True)
print('SPEC ALL')
batch_search_spec(
  df_spec=df_spec,
  df_clusters=df_clusters,
  suffix='spec+all',
  replace=True,
)
print('SPEC GALAXY + UNCLEAR')
batch_search_spec(
  df_spec=df_spec_GU,
  df_clusters=df_clusters,
  suffix='spec+G+U',
  replace=True,
)
print('SPEC GALAXY')
batch_search_spec(
  df_spec=df_spec_G,
  df_clusters=df_clusters,
  suffix='spec+G',
  replace=True,
)

SPEC ALL


A4059 (4.37 deg): 100%|██████████| 73/73 [02:29<00:00,  2.04s/it]            


SPEC GALAXY + UNCLEAR


A4059 (4.37 deg): 100%|██████████| 73/73 [02:00<00:00,  1.65s/it]            


SPEC GALAXY


A4059 (4.37 deg): 100%|██████████| 73/73 [01:21<00:00,  1.11s/it]            


In [14]:
df_clusters = load_table('tables/catalog_v4.csv')
print('Match ALL')
batch_crossmatch_spec_photo(
  df_clusters=df_clusters,
  variant='all',
  replace=True,
)
print('Match G+U')
batch_crossmatch_spec_photo(
  df_clusters=df_clusters,
  variant='G+U',
  replace=True,
)
print('Match G')
batch_crossmatch_spec_photo(
  df_clusters=df_clusters,
  variant='G',
  replace=True,
)

Match ALL


A194:   0%|          | 0/73 [00:00<?, ?it/s]

A4059: 100%|██████████| 73/73 [07:38<00:00,  6.28s/it]            


Match G+U


A4059: 100%|██████████| 73/73 [06:08<00:00,  5.04s/it]            


Match G


A4059: 100%|██████████| 73/73 [03:12<00:00,  2.64s/it]            


In [15]:
df_clusters = load_table('tables/catalog_v4.csv')
batch_prepare_tables(
  df_clusters=df_clusters,
  variant='all',
  replace=True,
)
batch_prepare_tables(
  df_clusters=df_clusters,
  variant='G',
  replace=True,
)
batch_prepare_tables(
  df_clusters=df_clusters,
  variant='G+U',
  replace=True,
)

A3158:  19%|█▉        | 14/73 [00:00<00:00, 69.32it/s]            

A4059: 100%|██████████| 73/73 [00:01<00:00, 60.60it/s]            
